# Agent Supervisor

Inspired by [LangChain](https://github.com/hwchase17/langchain) [Agent Supervisor](https://github.com/langchain-ai/langgraph/blob/main/examples/multi_agent/agent_supervisor.ipynb), The notebook showcases an AgentSupervisor powered by [Anthropic's Claude](https://www.anthropic.com/news/claude-3-family).


Below, we will create an agent group, with an agent supervisor to help delegate tasks.

![diagram](./img/supervisor-diagram.png)

Before we build, let's configure our environment:

In [1]:
# %%capture --no-stderr
# %pip install -U langchain langchain_anthropic langchain_experimental langgraph pandas

In [3]:
from getpass import getpass
import os


def _set_if_undefined(var: str):
  if not os.environ.get(var):
    os.environ[var] = getpass(f"Please provide your {var}")

_set_if_undefined("ANTHROPIC_API_KEY")
_set_if_undefined("TAVILY_API_KEY")

## Create tools

For this example, you will make an agent to do web research with a search engine. Define the tools they'll use below:

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults

tavily_tool = TavilySearchResults(max_results=5)

## Construct Graph

We're ready to start instantiating and invoking the graph using the AgentSupervisor class.

In [5]:
import sys
import logging
from langchain.callbacks.stdout import StdOutCallbackHandler

from slangchain.graphs.anthropic.schemas import NodeTool

from slangchain.graphs.anthropic.multi_agent.agent_supervisor import AgentSupervisor

stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.addHandler(stream_handler)

callbacks = [StdOutCallbackHandler()]

node_tools = [NodeTool.from_objs(
  tool = tavily_tool,
  name = "Researcher",
  description= "You are a web researcher."
)]

agent_super = AgentSupervisor.from_node_tools(
  node_tools = node_tools,
  early_stopping_method = "force",
  recursion_limit = 50,
  callbacks = callbacks,
  verbosity = True,
)
result = agent_super.invoke(input={"input": "Who is Apple."})
logger.info("RESULT: %s", result)



> Entering new AgentSupervisor chain...


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `bind_tools` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


{'supervisor': {'next': 'Researcher'}}
----


> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` id: `toolu_01VKJrAGhVocEz2jBu5vzZfP`, index: 0 input: `{'query': 'Apple company'}`
responded: [{'id': 'toolu_01VKJrAGhVocEz2jBu5vzZfP', 'input': {'query': 'Apple company'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}]

[{'url': 'https://www.apple.com/newsroom/', 'content': 'Building on this satellite infrastructure, Apple recently introduced Roadside Assistance via satellite to connect users to AAA if they have car trouble while outside of cellular and Wi-Fi coverage.2\xa0 Other safety capabilities that can provide assistance when it matters most include:\nPress Contacts\nBlair Ranger\nApple\nblair_ranger@apple.com\nApple Media Helpline\nmedia.help@apple.com\nApple Stories\nMore from Newsroom\nApple Footer\nShop and Learn\nShop and Learn\n+\nApple Wallet\nApple Wallet\n+\nAccount\nAccount\n+\nEntertainment\nEntertainment\n+\nApple Store\nApple Store\